<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/CNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import files
files.upload() 

Saving CreatePestImageData.py to CreatePestImageData.py
Saving SupportingFunctions.py to SupportingFunctions.py


{'CreatePestImageData.py': b'# -*- coding: utf-8 -*-\r\n"""createImageDataset.ipynb\r\n\r\nAutomatically generated by Colaboratory.\r\n\r\nOriginal file is located at\r\n    https://colab.research.google.com/drive/1FXiYKCQ-jtbL_O29BQb2Xy1bBd2inM9q\r\n"""\r\n\r\nfrom os import listdir\r\nimport numpy as np\r\nimport xml.etree.ElementTree as ET\r\nimport cv2\r\nimport tensorflow as tf \r\n\r\nclass createImageDataset(object):\r\n    \r\n    def __init__(self, class_map=None):\r\n        self.image_info = []\r\n        self.trainX = []\r\n        self.trainY = []\r\n        self.testX = []\r\n        self.testY = []\r\n            \r\n    def add_image_info(self , image_array , image_path , image_id , object_id , class_name , class_id):\r\n        \r\n        image_info_ = {\'object_id\': image_id,\r\n                       \'object_class\': class_name,\r\n                       \'image_path\': image_path,\r\n                       \'image_array\': image_array,\r\n                       \

In [1]:
import CreatePestImageData as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [8]:
support.Unzip('/content/GREENPATROL_DB_V1.0.zip','/content/Pests/Images')

In [9]:
support.fileCount('/content/Pests/Images')

2000


In [4]:
dataSet = Imagedataset.createImageDataset()

In [ ]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [2]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'wf', 3 : 'bt', 4 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object , 224,  '/content/Pests/Images/',  test_size=5000 )

In [6]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [29]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
# input layer
visible = Input(shape=(224,224,3))
# first feature extractor
conv1 = Conv2D(32, kernel_size=4, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
flat1 = Flatten()(pool1)
# second feature extractor
conv2 = Conv2D(16, kernel_size=8, activation='relu')(visible)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat2 = Flatten()(pool2)
# merge feature extractors
merge = concatenate([flat1, flat2])
# interpretation layer
hidden1 = Dense(10, activation='relu')(merge)
# prediction output
output = Dense(5, activation='softmax')(hidden1)
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 221, 221, 32) 1568        input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 217, 217, 16) 3088        input_13[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 110, 110, 32) 0           conv2d_9[0][0]                   
____________________________________________________________________________________________

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(16474, 224, 224, 3)
(16474,)
(5000, 224, 224, 3)
(5000,)


In [17]:
filepath="/content/drive/My Drive/Models/CNN3/weights-{epoch:02d}-{val_output_accuracy:.3f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='val_output_accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

Using TensorFlow backend.


In [31]:

from keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [32]:
model.fit(trainX,trainY,verbose=1,epochs=1)

Epoch 1/1
16474/16474 [==============================] - 335s 20ms/step - loss: 21.1666 - accuracy: 0.4862
